In [92]:
from DbConnector import DbConnector
from part2 import DbExecutor

In [93]:
# Set up the program
program = DbConnector()
executor = DbExecutor()

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('assignment2',)
-----------------------------------------------

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('assignment2',)
-----------------------------------------------



### Task 1

How many users, activities and trackpoints are there in the dataset (after it is inserted into the database)

In [94]:
# Get the number of Users
users = executor.get_table_size("User")
print(f"There are {users[0]} users")

# Get the number of Activities
activities = executor.get_table_size("Activity")
print(f"There are {activities[0]} activities")

# Get the number of TrackPoints
trackpoints = executor.get_table_size("TrackPoint")
print(f"There are {trackpoints[0]} trackpoints")


There are (182,) users
There are (14718,) activities


### Task 3

Find the top 15 users with the highest number of activities.

In [ ]:
query = "SELECT user_id FROM Activity GROUP BY user_id ORDER BY COUNT(*) DESC LIMIT 15"
program.cursor.execute(query)
rows = program.cursor.fetchall()
text = "Top 15 users with the highest number of activities:\n"

for i, val in enumerate(rows):
    text += f"Top {i+1}. {val[0]}\n"
print(text)

Top 15 users with the highest number of activities:
1. 163
2. 085
3. 153
4. 068
5. 167
6. 128
7. 062
8. 075
9. 126
10. 010
11. 052
12. 084
13. 179
14. 020
15. 112



### Task 4

Find all users who have taken a bus.

In [ ]:
# Get the users who have taken the bus
users = executor.get_user_taken_bus()
users = "".join([f'\n user: {user[0]}' for user in users[0]])
print(f"The user ids for user that have taken the bus: {users}")

The user ids for user that have taken the bus: 
 user: 010
 user: 020
 user: 052
 user: 053
 user: 058
 user: 062
 user: 064
 user: 065
 user: 067
 user: 068
 user: 069
 user: 073
 user: 075
 user: 078
 user: 080
 user: 081
 user: 082
 user: 084
 user: 085
 user: 088
 user: 091
 user: 092
 user: 096
 user: 098
 user: 100
 user: 101
 user: 102
 user: 104
 user: 105
 user: 108
 user: 110
 user: 111
 user: 112
 user: 114
 user: 124
 user: 125
 user: 126
 user: 128
 user: 129
 user: 138
 user: 139
 user: 141
 user: 147
 user: 153
 user: 154
 user: 161
 user: 163
 user: 167
 user: 174
 user: 175
 user: 179


### Task 6

Find activities that are registered multiple times. You should find the query even
if it gives zero result.

In [ ]:
query = "SELECT * FROM Activity AS a WHERE EXISTS (SELECT b.id FROM Activity AS b WHERE a.user_id = b.user_id AND a.transportation_mode = b.transportation_mode AND a.start_date_time = b.start_date_time AND a.end_date_time = b.end_date_time AND a.id != b.id)"
program.cursor.execute(query)
rows = program.cursor.fetchall()
print("Find activities that are registered multiple times:\n", rows)

### Task 7

a) Find the number of users that have started an activity in one day and ended the activity the next day.

In [ ]:
users = executor.get_user_activity_over_a_day()

num = set()
for user in users[0]:
    num.add(user[0])

count = len(num)

print(f"Number of users with activity that ends the next day: {count}")

Number of users with activity that ends the next day: 38


b) List the transportation mode, user id and duration for these activities.

In [ ]:
users = "".join([f'\n user: {user[0]} transportation mode: {user[1]} duration: {user[2]}' for user in users[0]])
print(f"The user who have an end date one day after the start day: {users}")
print(users[0])

The user who have an end date one day after the start day: 
 user: 010 transportation mode: train duration: 23:59:59
 user: 010 transportation mode: train duration: 11:13:11
 user: 010 transportation mode: train duration: 7:09:22
 user: 010 transportation mode: train duration: 12:13:22
 user: 010 transportation mode: train duration: 11:31:39
 user: 010 transportation mode: train duration: 9:20:15
 user: 010 transportation mode: train duration: 1:50:25
 user: 010 transportation mode: bus duration: 0:08:24
 user: 010 transportation mode: taxi duration: 0:42:47
 user: 010 transportation mode: taxi duration: 0:15:26
 user: 010 transportation mode: train duration: 0:30:26
 user: 010 transportation mode: subway duration: 0:08:56
 user: 010 transportation mode: taxi duration: 0:25:35
 user: 010 transportation mode: taxi duration: 0:30:20
 user: 010 transportation mode: train duration: 1:01:15
 user: 010 transportation mode: train duration: 0:30:11
 user: 010 transportation mode: train duratio

### Task 9

Find the top 15 users who have gained the most altitude meters.

In [ ]:
query = "SELECT user_id, sum(activity_altitude)*0.304 as altitude_in_meters FROM Activity JOIN (SELECT activity_id, SUM(difference) AS activity_altitude FROM (SELECT activity_id, altitude - LAG(altitude) OVER (PARTITION BY activity_id ORDER BY date_time) AS difference FROM TrackPoint) AS altitude_difference WHERE difference > 0 GROUP BY activity_id) AS difference_table ON Activity.id = difference_table.activity_id GROUP BY user_id ORDER BY altitude_in_meters DESC LIMIT 15"
program.cursor.execute(query)
rows = program.cursor.fetchall()
text = "Find the top 15 users who have gained the most altitude meters\n"
for i in rows:
    text += f"User: {i[0]}, Altitude in meters: {i[1]}"
print(text)

### Task 10

Find the users that have traveled the longest total distance in one day for each
transportation mode.

In [ ]:
# Closing the connection
program.close_connection()


-----------------------------------------------
Connection to 8.0.34-0ubuntu0.22.04.1 is closed


### Task 12

Find all the users who have registered transportation_mode and their most used transportation_mode

In [ ]:
query = "SELECT user_id, transportation_mode FROM ( SELECT user_id, transportation_mode, ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY COUNT(*) DESC, transportation_mode ASC) AS rownum FROM Activity JOIN User ON Activity.user_id = User.id WHERE transportation_mode IS NOT NULL AND User.has_labels=TRUE GROUP BY user_id, transportation_mode) AS activity_grouped WHERE rownum=1 ORDER BY user_id"
program.cursor.execute(query)
rows = program.cursor.fetchall()
print("Find all the users who have registered transportation_mode and their most used transportation_mode")